In [1]:
!sudo apt-get install build-essential swig
!pip install auto-sklearn==0.11.1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [2]:
!python -m pip install dask distributed --upgrade

Requirement already up-to-date: dask in /usr/local/lib/python3.6/dist-packages (2021.1.1)
Requirement already up-to-date: distributed in /usr/local/lib/python3.6/dist-packages (2021.1.1)


In [28]:
from autosklearn.regression import AutoSklearnRegressor

Fails after the initial instalation of auto-sklearn -> Restart runtime and run all

In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [11]:
X = pd.read_csv("x.csv", index_col=0)
y = pd.read_csv("y.csv", index_col=0)
w = pd.read_csv("w.csv", index_col=0)

In [12]:
w

,x
1,no program
2,no program
3,job search
4,job search
5,no program
...,...
62557,no program
62558,no program
62559,job search
62560,job search


In [33]:
w = w[(w['x']=='no program') | (w['x']=='job search')]
w = w.applymap(lambda x: 1 if x=='job search' else 0)

In [34]:
X = X.loc[w.index]
y = y.loc[w.index]

In [35]:
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(X, y, w, test_size=0.3, random_state=0)

In [37]:
transformer = StandardScaler()
transformer.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [39]:
X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)

In [51]:
automl = AutoSklearnRegressor(
    time_left_for_this_task=3600,
    per_run_time_limit=60,
    n_jobs=-1,
    memory_limit = 12288,
    include_preprocessors=["no_preprocessing"]
)

In [52]:
automl.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


AutoSklearnRegressor(dask_client=None,
                     delete_output_folder_after_terminate=True,
                     delete_tmp_folder_after_terminate=True,
                     disable_evaluator_output=False, ensemble_nbest=50,
                     ensemble_size=50, exclude_estimators=None,
                     exclude_preprocessors=None, get_smac_object_callback=None,
                     include_estimators=None,
                     include_preprocessors=['no_preprocessing'],
                     initial_configurations_via_metalearning=25,
                     load_models=None, logging_config=None,
                     max_models_on_disc=50, memory_limit=12288,
                     metadata_directory=None, metric=None, n_jobs=-1,
                     output_folder=None, per_run_time_limit=60,
                     resampling_strategy='holdout',
                     resampling_strategy_arguments=None, seed=1,
                     smac_scenario_args=None, time_left_for_this_task

In [53]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: bbf6ced57e1e1c7f87363e4e21fb3c58
  Metric: r2
  Best validation score: 0.167432
  Number of target algorithm runs: 162
  Number of successful target algorithm runs: 78
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 68
  Number of target algorithms that exceeded the memory limit: 16



In [54]:
print(automl.show_models())

[(0.360000, SimpleRegressionPipeline({'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'robust_scaler', 'feature_preprocessor:__choice__': 'no_preprocessing', 'regressor:__choice__': 'gradient_boosting', 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': 0.00016067231715123824, 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_max': 0.7388500574361156, 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_min': 0.051843851330249116, 'regressor:gradient_boosting:early_stop': 'off', 'regressor:gradient_boosting:l2_regularization': 2.1545439485286841e-07, 'regressor:gradient_boosting:learning_rate': 0.05267815748529996, '

In [55]:
from sklearn.metrics import r2_score, mean_squared_error, max_error
preds = automl.predict(X_test)
print(r2_score(y_test, preds))
print(mean_squared_error(y_test, preds))
print(max_error(y_test, preds))

0.15753455937509908
126.71849762876194
29.094211352989078
